In [1]:
import numpy as np
import pandas as pd
import json
import requests
from datetime import datetime, timedelta

import dataloader
import time
import helper
import pyodbc

In [2]:
apikey = helper.get_apikey()

In [3]:
datetime.today()
fetchdate = datetime.strftime(datetime.today() - timedelta(days=1), '%m_%d_%Y')
fetchdate

'12_22_2022'

In [3]:
movies = pd.read_json(f'http://files.tmdb.org/p/exports/movie_ids_{fetchdate}.json.gz', lines=True)
persons = pd.read_json(f'http://files.tmdb.org/p/exports/person_ids_{fetchdate}.json.gz', lines=True)
companies = pd.read_json(f'http://files.tmdb.org/p/exports/production_company_ids_{fetchdate}.json.gz', lines=True)
keywords = pd.read_json(f'http://files.tmdb.org/p/exports/keyword_ids_{fetchdate}.json.gz', lines=True)
collections = pd.read_json(f'http://files.tmdb.org/p/exports/collection_ids_{fetchdate}.json.gz', lines=True)
tvseries = pd.read_json(f'http://files.tmdb.org/p/exports/tv_series_ids_{fetchdate}.json.gz', lines=True)
tvnetwork = pd.read_json(f'http://files.tmdb.org/p/exports/tv_network_ids_{fetchdate}.json.gz', lines=True)

In [4]:
custom = {"id": "INT PRIMARY KEY", "original_title":"nvarchar(max)"}
dataloader.full_load_with_index(df=movies, tbl="MovieIds", custom=custom)

DataFrame full loaded to Table: MovieIds


In [5]:
dataurl = f'https://api.themoviedb.org/3/genre/movie/list?api_key={apikey}'
data = json.loads(requests.get(dataurl).text)
# df = pd.json_normalize(data)
genres = pd.DataFrame.from_dict(data['genres'])
custom = {"id": "INT PRIMARY KEY",}
dataloader.full_load_with_index(df=genres, tbl="Genres", custom=custom)

DataFrame full loaded to Table: Genres


In [6]:
custom = {"id": "INT PRIMARY KEY",}
dataloader.full_load_with_index(df=persons, tbl="Persons", custom=custom)
del persons

DataFrame full loaded to Table: Persons


In [4]:
custom = {"id": "INT PRIMARY KEY",}
dataloader.full_load_with_index(df=companies, tbl="Companies", custom=custom)

DataFrame full loaded to Table: Companies


In [8]:
custom = {"id": "INT PRIMARY KEY",}
dataloader.full_load_with_index(df=keywords, tbl="Keywords", custom=custom)

DataFrame full loaded to Table: Keywords


In [9]:
custom = {"id": "INT PRIMARY KEY",}
dataloader.full_load_with_index(df=collections, tbl="Collections", custom=custom)

DataFrame full loaded to Table: Collections


In [10]:
custom = {"id": "INT PRIMARY KEY",}
dataloader.full_load_with_index(df=tvseries, tbl="TvSeries", custom=custom)

DataFrame full loaded to Table: TvSeries


In [ ]:
custom = {"id": "INT PRIMARY KEY",}
dataloader.full_load_with_index(df=tvnetwork, tbl="TvNetwork", custom=custom)

In [18]:
cstring = helper.get_connstring()
with pyodbc.connect(cstring) as conn:
    # query = "select id from movieids"
    query = "select id from movieids where id not in (select id from moviesraw)"
    ids = pd.read_sql(query, conn)

/var/folders/74/ndv7v28d3078l2cq22t43pkc0000gn/T/ipykernel_78206/2405849386.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ids = pd.read_sql(query, conn)


In [19]:
x=2
y=1
total_chunks = len(ids)//1000 + 1
for chunk in np.array_split(ids['id'], len(ids)//1000 + 1):
    session = requests.Session()
    df_movies = None 
    dfs=[]
    for i in chunk:
        print(f"total: {total_chunks} chunk: {x}, Record: {y}") 
        dataurl = f'https://api.themoviedb.org/3/movie/{i}?api_key={apikey}'
        data = json.loads(session.get(dataurl, stream=False).text)
        df = pd.json_normalize(data)
        dfs.append(df)
        y+=1
    
    df_movies = pd.concat(dfs)
    df_movies = df_movies[['adult', 'backdrop_path', 'budget', 'genres', 'homepage', 'id',
                            'imdb_id', 'original_language', 'original_title', 'overview',
                            'popularity', 'poster_path', 'production_companies',
                            'production_countries', 'release_date', 'revenue', 'runtime',
                            'spoken_languages', 'status', 'tagline', 'title', 'video',
                            'vote_average', 'vote_count', 'belongs_to_collection.id']]

    df_movies.dropna(subset=['id'], axis=0, inplace=True)
    df_movies['id'] = pd.to_numeric(df_movies.id, errors='coerce').astype('int')
    df_movies['genres'] = df_movies['genres'].astype('str')
    df_movies['production_companies'] = df_movies['production_companies'].astype('str')
    df_movies['production_countries'] = df_movies['production_countries'].astype('str')
    df_movies['spoken_languages'] = df_movies['spoken_languages'].astype('str')
    
    df_movies['vote_average'] = pd.to_numeric(df_movies.vote_average, errors='coerce')
    df_movies['popularity'] = pd.to_numeric(df_movies.popularity, errors='coerce')
    df_movies['adult'] = df_movies['adult'].astype('bool')
    df_movies['video'] = df_movies['video'].astype('bool')
    
    
    # df_movies['vote_average'] = [format(i, '.3f') for i in df_movies['vote_average']]
    # df_movies['popularity'] = [format(i, '.3f') for i in df_movies['popularity']]
    
    # df_movies.to_csv('movies.csv', index=False)

            
    if x==1:   
        custom = {"id": "int PRIMARY KEY", "revenue": "bigint", "budget": "bigint"}
        dataloader.full_load_with_index(df=df_movies, tbl="MoviesRaw", custom=custom, fastexecute=False)
    else:
        # df_movies = pd.read_csv('movies.csv', engine='python')
        # print(df_movies.shape)
        # df_movies.dropna(subset=['id'], axis=0, inplace=True)
        # df_movies = df_movies.astype('object')
        dataloader.inc_load_with_index(df=df_movies, tbl="MoviesRaw", fastexecute=False)
    print(f"Loaded chunk: {x}")  
    x+=1
    session.close()
    del dfs
    del session
    

total: 74 chunk: 2, Record: 1
total: 74 chunk: 2, Record: 2
total: 74 chunk: 2, Record: 3
total: 74 chunk: 2, Record: 4
total: 74 chunk: 2, Record: 5
total: 74 chunk: 2, Record: 6
total: 74 chunk: 2, Record: 7
total: 74 chunk: 2, Record: 8
total: 74 chunk: 2, Record: 9
total: 74 chunk: 2, Record: 10
total: 74 chunk: 2, Record: 11
total: 74 chunk: 2, Record: 12
total: 74 chunk: 2, Record: 13
total: 74 chunk: 2, Record: 14
total: 74 chunk: 2, Record: 15
total: 74 chunk: 2, Record: 16
total: 74 chunk: 2, Record: 17
total: 74 chunk: 2, Record: 18
total: 74 chunk: 2, Record: 19
total: 74 chunk: 2, Record: 20
total: 74 chunk: 2, Record: 21
total: 74 chunk: 2, Record: 22
total: 74 chunk: 2, Record: 23
total: 74 chunk: 2, Record: 24
total: 74 chunk: 2, Record: 25
total: 74 chunk: 2, Record: 26
total: 74 chunk: 2, Record: 27
total: 74 chunk: 2, Record: 28
total: 74 chunk: 2, Record: 29
total: 74 chunk: 2, Record: 30
total: 74 chunk: 2, Record: 31
total: 74 chunk: 2, Record: 32
total: 74 chunk: 

In [7]:
df_movies.columns

Index(['adult', 'backdrop_path', 'belongs_to_collection', 'budget', 'genres',
       'homepage', 'id', 'imdb_id', 'original_language', 'original_title',
       'overview', 'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [5]:
df_movies.columns

Index(['adult', 'backdrop_path', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'belongs_to_collection.id', 'success',
       'status_code', 'status_message'],
      dtype='object')

In [11]:
pd.to_numeric(df_movies.vote_average, errors='coerce')

0    4.700
0    5.800
0    6.500
0    3.500
0    5.000
     ...  
0    5.643
0    5.100
0    0.000
0    5.000
0    6.800
Name: vote_average, Length: 728, dtype: float64

In [53]:
pd.to_numeric(df_test.id, errors='coerce')

0      224423.0
1      224438.0
2      224455.0
3      224473.0
4      224538.0
         ...   
964     58100.0
965     58131.0
966     58132.0
967     58149.0
968     58164.0
Name: id, Length: 952, dtype: float64

In [48]:
df_test.id.astype('str').str.isnumeric()

0      False
1      False
2      False
3      False
4      False
       ...  
964    False
965    False
966    False
967    False
968    False
Name: id, Length: 952, dtype: bool

In [17]:
df_movies['release_date'].sort_values() #= pd.to_datetime(df_movies['release_date']).dt.normalize()

1820             0
5073    1889-01-14
5575    1890-01-01
5576    1890-11-21
5165    1890-11-21
           ...    
5349           NaN
7532           NaN
7539           NaN
7571           NaN
7607           NaN
Name: release_date, Length: 7651, dtype: object

In [17]:
df_movies